In [ ]:
#establecemos  la conexion a drive para poder cargar los datos y guardar el modelo y el peso
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow-addons


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#librerias necesarias para que funciones la CNN
import sys
import os
from keras.preprocessing.image import ImageDataGenerator  
from keras.layers import Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, MaxPooling2D, Convolution2D, Flatten, Dropout
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

#declaramos los optimizadores a usar 
radam = tfa.optimizers.RectifiedAdam()
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

In [ ]:
device_list = tf.test.gpu_device_name()

In [ ]:
device_list

'/device:GPU:0'

In [ ]:
if device_list != '/device:GPU:0':
  raise SystemError('GPU device not found')
print ('Found GPU at: {}'.format(device_list))

Found GPU at: /device:GPU:0


In [ ]:
#funciones para las métricas  que vamos a usar 

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
# aqui matamos/ cerramos la sesión en caso de que estuviese trabajando en segundo plano
K.clear_session()

In [ ]:
#cargamos los set de datos 
data_entrenamiento = ('/content/drive/MyDrive/SIS360/entrenamiento')
data_validacion = ('/content/drive/MyDrive/SIS360/validacion')

In [ ]:
# variables con las que vamos a trabajar para poder entrenar la CNN
epocas= 55
longitud, altura = 150, 150
batch_size = 32
pasos_validacion = 300
clases = 4
learning_rate = 0.0005

In [ ]:
# establecemos como vamos a trabajar para poder manipular las imagenes
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True)

In [ ]:
# establecemos el como va cargar los datos y el modo de clase con el que vamos a entrenar 
validacion_datagen = ImageDataGenerator(rescale=1. / 255)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size = batch_size,
    class_mode= 'categorical' 
)

Found 2861 images belonging to 4 classes.


In [ ]:
# lo mismo que el anterior pero para los datos de validación 
imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size = (altura, longitud),
    batch_size= batch_size,
    class_mode= 'categorical'
)

Found 466 images belonging to 4 classes.


In [ ]:
# empezamos a trabajar con las convoluciones cargando parametros 

cnn = Sequential()

cnn.add(keras.layers.Conv2D(
            512,
            (3, 3),
            padding="same", input_shape=(longitud, altura, 3),
            activation=keras.layers.LeakyReLU()
        ))


cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.AveragePooling2D())

cnn.add(keras.layers.Conv2D(
            256,
            (3, 3),
            padding="same", input_shape=(longitud, altura, 3),
            activation=keras.layers.LeakyReLU()
        ))

cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.AveragePooling2D())

cnn.add(Flatten())
cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.Dense(512, activation=keras.layers.LeakyReLU()))
cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.Dense(256, activation=keras.layers.LeakyReLU()))
cnn.add(keras.layers.BatchNormalization())
cnn.add(Dropout(0.4))
cnn.add(Dense(clases, activation='softmax'))


In [ ]:
#compila la cnn con los parametros establecidos y nos da el porcentaje de precisión
cnn.compile(loss = 'categorical_crossentropy',
            optimizer = ranger,
            metrics = ["accuracy", f1_m, precision_m, recall_m])
cnn.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 512)     14336     
                                                                 
 batch_normalization (BatchN  (None, 150, 150, 512)    2048      
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 75, 75, 512)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 256)       1179904   
                                                                 
 batch_normalization_1 (Batc  (None, 75, 75, 256)      1024      
 hNormalization)                                                 
                                                        

In [ ]:
#entrenamos el algoritmo y lo alimentamos con el entrenamiento
cnn.fit(
    imagen_entrenamiento,
    epochs=epocas,
    validation_data=imagen_validacion,
    validation_steps=pasos_validacion)

Epoch 1/55
90/90 [==============================] - ETA: 0s - loss: 1.4639 - accuracy: 0.5012 - f1_m: 0.4861 - precision_m: 0.5300 - recall_m: 0.4508

90/90 [==============================] - 232s 2s/step - loss: 1.4639 - accuracy: 0.5012 - f1_m: 0.4861 - precision_m: 0.5300 - recall_m: 0.4508 - val_loss: 1.3328 - val_accuracy: 0.3155 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/55
90/90 [==============================] - 67s 743ms/step - loss: 1.0909 - accuracy: 0.5890 - f1_m: 0.5804 - precision_m: 0.6247 - recall_m: 0.5431
Epoch 3/55
90/90 [==============================] - 63s 699ms/step - loss: 1.0028 - accuracy: 0.6225 - f1_m: 0.6092 - precision_m: 0.6466 - recall_m: 0.5770
Epoch 4/55
90/90 [==============================] - 64s 703ms/step - loss: 0.9324 - accuracy: 0.6292 - f1_m: 0.6192 - precision_m: 0.6533 - recall_m: 0.5894
Epoch 5/55
90/90 [==============================] - 64s 707ms/step - loss: 0.8773 - accuracy: 0.6536 - f1_m: 0.6479 - precision_m: 0.6806 - recall_m: 0.6195
Epoch 6/55
90/90 [==============================] - 63s 698ms/step - loss: 0.8257 - accuracy: 0.6669 - f1_

In [ ]:
#guardamos en una carpeta los pesos y la estructura del modelo
target_dir = ('/content/drive/MyDrive/SIS360/modelo')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('/content/drive/MyDrive/SIS360/modelo/modelo.h5')
cnn.save_weights('/content/drive/MyDrive/SIS360/modelo/pesos.h5')

In [ ]:
# se instala tensorflowjs para exportar el modelo y poderlo usar en una página web
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# crear carpeta de salida para exportar a js

target_dir1 = ('/content/drive/MyDrive/carpeta_salida')
if not os.path.exists(target_dir1):
  os.mkdir(target_dir1)


In [ ]:
# realizamos la exportación a la carpeta de salida 
!tensorflowjs_converter --input_format keras /content/drive/MyDrive/SIS360/modelo/modelo.h5 /content/drive/MyDrive/carpeta_salida

2023-02-09 00:34:14.971838: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-09 00:34:14.971994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-09 00:34:14.972014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
#confirmamos que se haya realizado

!ls carpeta_salida 

ls: cannot access 'carpeta_salida': No such file or directory
